Playing with Hooks
==================

Setting hooks is the main interface with an execution and an exploration to perform
user-defined actions. Tritondse enables hooking the following events:

* address reached
* instruction executed *(all of them)*
* memory address read or written
* register read or written
* function reached *(from its name)*
* end of an execution
* thread context switch
* new input creation *(before it gets appended in the pool of seeds)*

The library introduces a `CallbackManager` object which enables registering
callbacks. A `SymbolicExecutor` do contains such object. In the case of a
`SymbolicExplorator` it also contains a callback_manager instance, but this
one will be transmitted to all subsequent `SymbolicExecutor` instances.


For this notebook we are going to reuse a `SymbolicExecutor` and hooking
various events of the run.

In [8]:
from triton import Instruction
from tritondse import SymbolicExecutor, Config, Seed, Program, ProcessState

p = Program("crackme_xor")
config = Config(symbolize_argv=True, pipe_stdout=False)
seed = Seed(b"./crackme_xor AAAAAAAAAAAA")

executor = SymbolicExecutor(config, seed)
executor.load_program(p)

## I. Instruction hooking

Instruction hooking enables hooking the execution of every instruction executed regardless of theirs address etc.

The signature for an instruction hook is the following:

```python
Callable[['SymbolicExecutor', ProcessState, Instruction], None]
```

We can use it to print every instructions executed:

In [9]:
def trace_inst(exec: SymbolicExecutor, pstate: ProcessState, inst: Instruction):
    print(f"[tid:{inst.getThreadId()}] 0x{inst.getAddress():x}: {inst.getDisassembly()}")

executor.callback_manager.register_post_instuction_callback(trace_inst)

In [10]:
executor.run()

[tid:0] 0x400460: xor ebp, ebp
[tid:0] 0x400462: mov r9, rdx
[tid:0] 0x400465: pop rsi
[tid:0] 0x400466: mov rdx, rsp
[tid:0] 0x400469: and rsp, 0xfffffffffffffff0
[tid:0] 0x40046d: push rax
[tid:0] 0x40046e: push rsp
[tid:0] 0x40046f: mov r8, 0x400680
[tid:0] 0x400476: mov rcx, 0x400610
[tid:0] 0x40047d: mov rdi, 0x4005b3
[tid:0] 0x400484: call 0x400440
[tid:0] 0x400440: jmp qword ptr [rip + 0x200bda]
[tid:0] 0x4005b3: push rbp
[tid:0] 0x4005b4: mov rbp, rsp
[tid:0] 0x4005b7: sub rsp, 0x20
[tid:0] 0x4005bb: mov dword ptr [rbp - 0x14], edi
[tid:0] 0x4005be: mov qword ptr [rbp - 0x20], rsi
[tid:0] 0x4005c2: cmp dword ptr [rbp - 0x14], 2
[tid:0] 0x4005c6: je 0x4005cf
[tid:0] 0x4005cf: mov rax, qword ptr [rbp - 0x20]
[tid:0] 0x4005d3: add rax, 8
[tid:0] 0x4005d7: mov rax, qword ptr [rax]
[tid:0] 0x4005da: mov rdi, rax
[tid:0] 0x4005dd: call 0x400556
[tid:0] 0x400556: push rbp
[tid:0] 0x400557: mov rbp, rsp
[tid:0] 0x40055a: mov qword ptr [rbp - 0x18], rdi
[tid:0] 0x40055e: mov dword ptr [

We could also have used `register_pre_instruction_callback` but at this point the `Instruction` object is not yet decoded so it prevent getting its disassembly.

## II. Address/Function hooking

We can hook any address and perform any associated action.  
We can also hook any function as long as the symbol is set.
They both have the same signature:

```python
Callable[['SymbolicExecutor', ProcessState, Addr], None]
```

For the purpose of the challenge let's hook the compare instruction and patch de ZF flag to force looping.
Let's also hook the `puts` function to print the string given to each call.

In [14]:
def hook_cmp(exec: SymbolicExecutor, pstate: ProcessState, addr: int):
    print(f"{pstate.cpu.al} - {pstate.cpu.cl}")
    pstate.cpu.zf = 1
    #exec.abort()

def hook_puts(exec: SymbolicExecutor, pstate: ProcessState, routine: str, addr: int):
    s = pstate.get_memory_string(pstate.get_argument_value(0))
    print(f"puts: {s}")

In [17]:
executor = SymbolicExecutor(config, seed)
executor.load_program(p)

# Remove trace printing callback
executor.callback_manager.reset()
executor.callback_manager.register_post_addr_callback(0x0400597, hook_cmp)
executor.callback_manager.register_post_imported_routine_callback("puts", hook_puts)

In [18]:
executor.run()

49 - 21
62 - 21
61 - 21
38 - 21
49 - 21
puts: Win


We did not really win where as we forced the ZF flag, but we have encoded values on wich
the comparison is made.

## III. Solving queries

We can modify our hook to directly solve by SMT what shall be the appropriate value of CL in order to match the comparison.

In [20]:
from tritondse.types import SolverStatus

def hook_cmp2(exec: SymbolicExecutor, pstate: ProcessState, addr: int):
    # CL contains the input of the user (hashed)
    
    # retrieve the symbolic value of both characters
    sym_al = pstate.read_symbolic_register(pstate.registers.al)
    sym_cl = pstate.read_symbolic_register(pstate.registers.cl)
    
    # Solve the constraint such that one match the other
    status, model = pstate.solve(sym_al.getAst() == sym_cl.getAst())
    
    # If formula is SAT retrieve input values
    if status == SolverStatus.SAT:
        # Retrieve value of the input variable involved in the cl value here (shall be only one here)
        var_values = pstate.get_expression_variable_values_model(sym_cl, model)
        for var, value in var_values.items():
            print(f"{var}: {chr(value)}")
    else:
        print(status.name)
    
    pstate.cpu.zf = 1

In [21]:
executor = SymbolicExecutor(config, seed)
executor.load_program(p)

executor.callback_manager.reset()
executor.callback_manager.register_post_addr_callback(0x0400597, hook_cmp2)

In [22]:
executor.run()

argv[1][0]:8: e
argv[1][1]:8: l
argv[1][2]:8: i
argv[1][3]:8: t
argv[1][4]:8: e


## IV. Hooking exploration events

We can similarly put callbacks on a `SymbolicExplorator`. In this case, the callback manager
will be shared among all the `SymbolicExecutor` instances. Let's hook every iteration to print
some statistics:

In [24]:
from tritondse import SymbolicExplorator

def pre_exec_hook(se: SymbolicExecutor, state: ProcessState):
    print(f"input: {se.seed.hash}  ", end="")

def post_exec_hook(se: SymbolicExecutor, state: ProcessState):
    print(f"status:{se.seed.status.name}   [exitcode:{se.exitcode}]")

dse = SymbolicExplorator(Config(symbolize_argv=True), p)
dse.add_input_seed(Seed(b"./crackme AAAAAAAAAAAAAAA"))

dse.callback_manager.register_pre_execution_callback(pre_exec_hook)
dse.callback_manager.register_post_execution_callback(post_exec_hook)

dse.explore()

input: 78fd4aa0744187fcda352908d6263e3b  status:OK_DONE   [exitcode:0]
input: ba39b0af614b34616b62e732d2cd2c3f  status:OK_DONE   [exitcode:0]


<ExplorationStatus.IDLE: 2>